In [13]:
import featuretools as ft
import pandas as pd
import numpy as np
import utils_instacart as utils
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from dldb import DLDB
import os
from keras.callbacks import EarlyStopping
ft.__version__

'0.1.18'

In [2]:
es = utils.load_entityset('partitioned_data/part_0/')

In [3]:
cutoff_time = pd.Timestamp('March 1, 2015')
training_window = ft.Timedelta("60 days")

In [4]:
fm = utils.denormalize_entityset(es, cutoff_time, training_window)
fm.sort_index(inplace=True)

In [5]:
label_times = utils.make_labels(es,
                                product_name="Banana",
                                cutoff_time=cutoff_time,
                                prediction_window=ft.Timedelta("4 weeks"),
                                training_window=training_window)
labels = label_times.set_index('user_id').sort_index()['label']

In [6]:
dl_model = DLDB(
    regression=False,
    classes=[False, True],
    recurrent_layer_sizes=(32, 32),
    dense_layer_sizes=(32, 32),
    dropout_fraction=0.2,
    recurrent_dropout_fraction=0.1,
    categorical_embedding_size=20,
    categorical_max_vocab=12)
# TODO: cheating a bit, put back in CV later
#dl_model.compile(fm, top_features)
dl_model.compile(fm, categorical_feature_names=[c for c in fm.columns if c != 'reordered'])

In [15]:
cv_score = []
n_splits = 3
test_frac = 0.1
# Use 10% of data as testing set, but only run 3 rounds of cross-validation
# (because they take a while)
splitter = StratifiedKFold(n_splits=int(1/test_frac), shuffle=True)

for i, train_test_index in enumerate(splitter.split(labels, labels)):
    train_labels = labels.iloc[train_test_index[0]]
    test_labels = labels.iloc[train_test_index[1]]
    train_fm = fm.loc[(train_labels.index, slice(None)), :]
    test_fm = fm.loc[(test_labels.index, slice(None)), :]

    dl_model.fit(
        train_fm, train_labels,
        validation_split=0.1,
        epochs=100,
        batch_size=32,
        callbacks=[EarlyStopping()])
    
    predictions = dl_model.predict(test_fm)
    cv_score.append(roc_auc_score(test_labels, predictions))
    if i == n_splits - 1:
        break
mean_score = np.mean(cv_score)
stderr = 2 * (np.std(cv_score) / np.sqrt(n_splits))

print("AUC %.2f +/- %.2f" % (mean_score, stderr))

Transforming input matrix into numeric sequences
Fitting Keras model
Train on 678 samples, validate on 76 samples
Epoch 1/100
678/678 [==============================] - 14s 21ms/step - loss: 0.0847 - val_loss: 1.0136
Epoch 2/100
678/678 [==============================] - 15s 22ms/step - loss: 0.0859 - val_loss: 0.9736
Epoch 3/100
678/678 [==============================] - 14s 21ms/step - loss: 0.0726 - val_loss: 0.9404
Epoch 4/100
678/678 [==============================] - 14s 21ms/step - loss: 0.0691 - val_loss: 1.0074
Transforming input matrix into numeric sequences
Predicting using Keras model
Transforming outputs
Transforming input matrix into numeric sequences
Fitting Keras model
Train on 679 samples, validate on 76 samples
Epoch 1/100
679/679 [==============================] - 14s 21ms/step - loss: 0.0594 - val_loss: 0.9786
Epoch 2/100
679/679 [==============================] - 14s 20ms/step - loss: 0.0757 - val_loss: 0.9185
Epoch 3/100
679/679 [==============================] - 